In [1]:
import pandas as pd
import numpy as np
import sys

import matplotlib.pyplot as plt
# from bertopic import BERTopic
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [3]:
user_resp = pd.read_csv('../dataset/user_response.csv')
softmax_score_df = pd.read_csv('../dataset/user_resp_MFT_all_softmax.csv')

In [4]:
user_resp.columns, user_resp.shape, softmax_score_df.shape

(Index(['Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1', 'tweet_id',
        'conversation_id', 'author_id_x', 'created_at_x', 'geo_x', 'lang_x',
        'like_count_x', 'quote_count_x', 'reply_count_x', 'retweet_count_x',
        'source_x', 'text_x', 'anger', 'joy', 'optimism', 'sadness',
        'tweet_id_y', 'author_id_y', 'created_at_y', 'geo_y', 'lang_y',
        'like_count_y', 'quote_count_y', 'reply_count_y', 'retweet_count_y',
        'source_y', 'text_y', 'publication', 'topics_x', 'topic_labels',
        'theme', 'vad_pos_senti', 'vad_neu_senti', 'vad_neg_senti', 'trump_neg',
        'trump_neu', 'trump_pos', 'biden_neg', 'biden_neu', 'biden_pos',
        'trump_flag', 'biden_flag', 'tweet_freq_y', 'neg_senti', 'neu_senti',
        'pos_senti', 'topics_y', 'subtopic', 'topic_ids', 'subtopic.1',
        'trump_pos_new', 'trump_neg_new', 'trump_neu_new', 'biden_pos_new',
        'biden_neg_new', 'biden_neu_new', 'Trump_flag', 'Biden_flag',
        'Trump_Biden_flag', 'topic_id',

In [5]:
user_resp.theme.value_counts()

other            2668201
voter             397146
international     271971
covid             247437
racism             99502
economy            75805
court              56500
national           55081
impeachment        52294
georgia            41024
school             37765
climate            27851
healthcare          8892
Name: theme, dtype: int64

In [6]:
user_resp_df = user_resp[['tweet_id', 'conversation_id', 'author_id_x', 'publication', 'topics_x', 'theme', 'topic_labels', 'trump_pos_new', 'trump_neg_new', 'trump_neu_new', 'biden_pos_new', 'biden_neg_new', 
                          'biden_neu_new', 'Trump_flag', 'Biden_flag', 'Trump_Biden_flag', 'topic_id', 'clean_text', 'pred_labels']].merge(softmax_score_df, on = 'tweet_id', how = 'inner')

In [8]:
user_resp_df.shape, user_resp_df.columns

((4039474, 32),
 Index(['tweet_id', 'conversation_id', 'author_id_x', 'publication', 'topics_x',
        'theme', 'topic_labels', 'trump_pos_new', 'trump_neg_new',
        'trump_neu_new', 'biden_pos_new', 'biden_neg_new', 'biden_neu_new',
        'Trump_flag', 'Biden_flag', 'Trump_Biden_flag', 'topic_id',
        'clean_text', 'pred_labels_x', 'Unnamed: 0', '0', '1', '2', '3', '4',
        '5', '6', '7', '8', '9', '10', 'pred_labels_y'],
       dtype='object'))

In [9]:
# user_resp_df['topic_labels'] = user_resp.topic_labels

In [10]:
R_user_resp = user_resp_df[((user_resp_df['publication'] == 'Breitbart News') | (user_resp_df['publication'] == 'Fox News'))]
L_user_resp = user_resp_df[((user_resp_df['publication'] == 'CNN') | (user_resp_df['publication'] == 'The Washington Post'))]
C_user_resp = user_resp_df[((user_resp_df['publication'] == 'Business Insider') | (user_resp_df['publication'] ==  'USA Today'))]

In [11]:
from pingouin import kruskal

def get_KW_stat(emot1, emot2):
    
    df1 = pd.DataFrame(emot1.values, columns = ['senti'])
    df1['type_flag'] = 1

    df2 = pd.DataFrame(emot2.values, columns = ['senti'])
    df2['type_flag'] = 2
    
    temp_df = pd.concat((df1,df2), axis = 0)
    kwTest = kruskal(temp_df, dv='senti', between='type_flag')
    H = kwTest.H[0]
    p = kwTest['p-unc'][0]
    dof = kwTest['ddof1'][0]
    n = temp_df.shape[0]
    esq = H * (n + 1)/(n**2 - 1)
    return H, p, dof, esq


def get_pos_neg_mentions(df, entity):
    
    entity_mentions = df[df[entity + '_flag'] == True]
    pos_mean = entity_mentions[entity + '_pos'].mean()
    neu_mean = entity_mentions[entity + '_neu'].mean()
    neg_mean = entity_mentions[entity + '_neg'].mean()
    pos_entity_mentions = entity_mentions[((entity_mentions[entity + '_pos_new'] > pos_mean))]
    neg_entity_mentions = entity_mentions[((entity_mentions[entity + '_neg_new'] > neg_mean))]
    
    return pos_entity_mentions, neg_entity_mentions

def get_significance_stats(dist1, dist2, pub, entity, emotion):

    emot1_shapiro = stats.shapiro(dist1)
    emot2_shapiro = stats.shapiro(dist2)
    emot1_shapiro = (emot1_shapiro.statistic, emot1_shapiro.pvalue)
    emot2_shapiro = (emot2_shapiro.statistic, emot2_shapiro.pvalue)
    H, p, dof, es = get_KW_stat(dist1, dist2)
    # t, p , dof = welch_ttest(emot1, emot2)
    res = [entity, pub, emotion, dist1.shape[0], dist2.shape[0], emot1_shapiro[0], emot1_shapiro[1], emot2_shapiro[0], emot2_shapiro[1], p, H, dof, es]
    # print(res)
    return res

def get_mean_emotions(df):
    
    return df['anger'].mean(), df['joy'].mean(), df['optimism'].mean(), df['sadness'].mean()

def get_mean_emotions_per_sentiment(df, entity):
    
    entity_mentions = df[df[entity.title() + '_flag'] == True]
    pos_mean = entity_mentions[entity + '_pos_new'].mean()
    neu_mean = entity_mentions[entity + '_neu_new'].mean()
    neg_mean = entity_mentions[entity + '_neg_new'].mean()
    pos_entity_mentions = entity_mentions[((entity_mentions[entity + '_pos_new'] > pos_mean * 1.5))]
    neg_entity_mentions = entity_mentions[((entity_mentions[entity + '_neg_new'] > neg_mean * 1.5))]
    
#     print(entity, pos_entity_mentions.shape, pos_entity_mentions_.shape)
#     print(entity, neg_entity_mentions.shape, neg_entity_mentions_.shape)
    
    # pos_tweet_emot = get_mean_emotions(pos_entity_mentions)
    # neg_tweet_emot = get_mean_emotions(neg_entity_mentions)
    
    return pos_entity_mentions, neg_entity_mentions

In [12]:
user_resp_df.head()

,tweet_id,conversation_id,author_id_x,publication,topics_x,trump_pos_new,trump_neg_new,trump_neu_new,biden_pos_new,biden_neg_new,...,1,2,3,4,5,6,7,8,9,10
0,1.353534e+18,1.353481e+18,1.140389e+09,The Washington Post,-1.0,NaN,NaN,NaN,0.584414,0.183797,...,0.026935,0.024551,0.025086,0.303849,0.010861,0.007318,0.441399,0.001886,0.006512,0.002431
1,1.353528e+18,1.353481e+18,9.446603e+17,The Washington Post,-1.0,NaN,NaN,NaN,0.584414,0.183797,...,0.026043,0.000524,0.951857,0.001575,0.007074,0.004732,0.001196,0.000186,0.003072,0.002708
2,1.353526e+18,1.353481e+18,9.446603e+17,The Washington Post,-1.0,NaN,NaN,NaN,0.584414,0.183797,...,0.000366,0.000966,0.990159,0.000397,0.000267,0.004682,0.000405,0.000486,0.001621,0.000540
3,1.353523e+18,1.353481e+18,9.446603e+17,The Washington Post,-1.0,NaN,NaN,NaN,0.584414,0.183797,...,0.003858,0.000776,0.978319,0.003051,0.001301,0.001638,0.003831,0.001216,0.004693,0.000639
4,1.353525e+18,1.353481e+18,3.445109e+08,The Washington Post,-1.0,NaN,NaN,NaN,0.584414,0.183797,...,0.006691,0.000471,0.946666,0.000737,0.008338,0.019390,0.001578,0.000347,0.003994,0.009065


### Left-vs-Right (user response) comparison [overall]

In [12]:
# emotions = ['anger', 'joy', 'optimism', 'sadness']
# topics = set(R_user_resp.topics_x).intersection(set(L_user_resp.topics_x))
sstat = []
    
for moral_cat in range(11):

    # n = min(l_df_neg.shape[0], r_df_neg.shape[0])
    dist1 = L_user_resp[str(moral_cat)].dropna()
    dist2 = R_user_resp[str(moral_cat)].dropna()
    # dist2 = C_user_resp[str(moral_cat)].dropna()
    

#             n = min(dist1.shape[0], dist2.shape[0])

#             dist1 = dist1.sample(n)
#             dist2 = dist2.sample(n)
    # print(dist1.shape, dist2.shape)

    # print(stats.shapiro(dist1), stats.shapiro(dist2))
    # sys.exit()
    if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

        res = get_significance_stats(dist1, dist2, None, moral_cat, moral_cat)
        # print(res)
        sstat.append(res)

    else:
        print(top, senti, dist1.shape, dist2.shape)
    # sys.exit()
# print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'Topic', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(4)
stats_df

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,0,None,0,2755152,1123219,0.1673,0.0,0.1522,0.0,0.000,4399.6482,1,0.0011
1,1,None,1,2755152,1123219,0.8524,0.0,0.8441,0.0,0.000,284.2714,1,0.0001
2,2,None,2,2755152,1123219,0.6986,0.0,0.6900,0.0,0.000,646.9239,1,0.0002
3,3,None,3,2755152,1123219,0.3364,0.0,0.2976,0.0,0.000,1381.4890,1,0.0004
4,4,None,4,2755152,1123219,0.8850,0.0,0.8868,0.0,0.000,2078.5499,1,0.0005
5,5,None,5,2755152,1123219,0.2571,0.0,0.2507,0.0,0.000,4298.6680,1,0.0011
6,6,None,6,2755152,1123219,0.6864,0.0,0.6200,0.0,0.000,1741.3146,1,0.0004
7,7,None,7,2755152,1123219,0.5362,0.0,0.5311,0.0,0.000,268.4672,1,0.0001
8,8,None,8,2755152,1123219,0.3104,0.0,0.2893,0.0,0.156,2.0124,1,0.0000
9,9,None,9,2755152,1123219,0.1794,0.0,0.1686,0.0,0.000,315.7650,1,0.0001


In [14]:
mft_id_to_label_mapping = pd.read_csv('../dataset/MFT_label_to_id_mapping_new.csv')

mft_id_to_label_mapping = mft_id_to_label_mapping.rename(columns = {'label_id': 'senti'})
mft_id_to_label_mapping

stats_df.merge(mft_id_to_label_mapping, on = 'senti').to_csv('results/statistical_tests_L_vs_R_user_resp_MF.csv')

### Left-vs-Right (user response) comparison [topic-wise]

In [15]:
# emotions = ['anger', 'joy', 'optimism', 'sadness']
topics = set(R_user_resp.theme).intersection(set(L_user_resp.theme))
sstat = []
# for ent in ['biden', 'trump']:
    
    # l_df_pos, l_df_neg, _, _ = get_mean_emotions_per_sentiment(L_user_resp[L_user_resp[ent + '_flag'] == True], ent)
    # r_df_pos, r_df_neg, _, _ = get_mean_emotions_per_sentiment(R_user_resp[R_user_resp[ent + '_flag'] == True], ent)
    
for top in topics:

    dataL = L_user_resp[L_user_resp.theme == top]
    dataR = R_user_resp[R_user_resp.theme == top]

    for moral_cat in range(11):

        # n = min(l_df_neg.shape[0], r_df_neg.shape[0])
        dist1 = dataL[str(moral_cat)].dropna()
        dist2 = dataR[str(moral_cat)].dropna()

#             n = min(dist1.shape[0], dist2.shape[0])

#             dist1 = dist1.sample(n)
#             dist2 = dist2.sample(n)
        # print(dist1.shape, dist2.shape)

        # print(stats.shapiro(dist1), stats.shapiro(dist2))
        # sys.exit()
        if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

            res = get_significance_stats(dist1, dist2, top, moral_cat, moral_cat)
            # print(res)
            sstat.append(res)

        else:
            print(top, senti, dist1.shape, dist2.shape)
        # sys.exit()
    # print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'Topic', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,0,georgia,0,24989,14213,0.143,0.0,0.132,0.0,0.183,1.773,1,0.000
1,1,georgia,1,24989,14213,0.863,0.0,0.874,0.0,0.000,42.097,1,0.001
2,2,georgia,2,24989,14213,0.747,0.0,0.754,0.0,0.016,5.760,1,0.000
3,3,georgia,3,24989,14213,0.256,0.0,0.220,0.0,0.337,0.921,1,0.000
4,4,georgia,4,24989,14213,0.871,0.0,0.873,0.0,0.948,0.004,1,0.000
5,5,georgia,5,24989,14213,0.283,0.0,0.280,0.0,0.000,12.669,1,0.000
6,6,georgia,6,24989,14213,0.633,0.0,0.646,0.0,0.002,9.241,1,0.000
7,7,georgia,7,24989,14213,0.528,0.0,0.558,0.0,0.000,38.628,1,0.001
8,8,georgia,8,24989,14213,0.319,0.0,0.311,0.0,0.617,0.251,1,0.000
9,9,georgia,9,24989,14213,0.170,0.0,0.162,0.0,0.060,3.535,1,0.000


### Left-vs-Right (user response) comparison [entity-wise]

In [143]:
# emotions = ['anger', 'joy', 'optimism', 'sadness']
# topics = set(R_user_resp.theme).intersection(set(L_user_resp.theme))
sstat = []
    
for ent in ['biden', 'trump']:

    # l_df_pos, l_df_neg, _, _ = get_mean_emotions_per_sentiment(L_user_resp[L_user_resp[ent + '_flag'] == True], ent)
    # r_df_pos, r_df_neg, _, _ = get_mean_emotions_per_sentiment(R_user_resp[R_user_resp[ent + '_flag'] == True], ent)
    
    dataL = L_user_resp[L_user_resp[ent.title() + '_flag'] == True]
    dataR = R_user_resp[R_user_resp[ent.title() + '_flag'] == True]

    for moral_cat in range(11):

        # n = min(l_df_neg.shape[0], r_df_neg.shape[0])
        dist1 = dataL[str(moral_cat)].dropna()
        dist2 = dataR[str(moral_cat)].dropna()

#             n = min(dist1.shape[0], dist2.shape[0])

#             dist1 = dist1.sample(n)
#             dist2 = dist2.sample(n)
        # print(dist1.shape, dist2.shape)

        # print(stats.shapiro(dist1), stats.shapiro(dist2))
        # sys.exit()
        if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

            res = get_significance_stats(dist1, dist2, ent, moral_cat, moral_cat)
            # print(res)
            sstat.append(res)

        else:
            print(top, senti, dist1.shape, dist2.shape)
        # sys.exit()
    # print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'Topic', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,0,biden,0,759457,563259,0.436,0.0,0.412,0.0,0.000,78.676,1,0.000
1,1,biden,1,759457,563259,0.533,0.0,0.536,0.0,0.000,94.269,1,0.000
2,2,biden,2,759457,563259,0.131,0.0,0.109,0.0,0.000,1378.514,1,0.001
3,3,biden,3,759457,563259,0.752,0.0,0.743,0.0,0.000,21.831,1,0.000
4,4,biden,4,759457,563259,0.158,0.0,0.145,0.0,0.000,880.017,1,0.001
5,5,biden,5,759457,563259,0.409,0.0,0.385,0.0,0.000,130.970,1,0.000
6,6,biden,6,759457,563259,0.307,0.0,0.286,0.0,0.000,82.859,1,0.000
7,7,biden,7,759457,563259,0.325,0.0,0.296,0.0,0.000,726.298,1,0.001
8,8,biden,8,759457,563259,0.060,0.0,0.055,0.0,0.000,750.607,1,0.001
9,9,biden,9,759457,563259,0.188,0.0,0.172,0.0,0.000,369.721,1,0.000


### Left-vs-Right (user response) comparison [positive and negative entity mentions]

In [33]:
# emotions = ['anger', 'joy', 'optimism', 'sadness']
topics = set(R_user_resp.topics_x).intersection(set(L_user_resp.topics_x))
sstat = []
    
for ent in ['biden', 'trump']:
    
    l_df_pos, l_df_neg = get_mean_emotions_per_sentiment(L_user_resp[L_user_resp[ent.title() + '_flag'] == True], ent)
    r_df_pos, r_df_neg = get_mean_emotions_per_sentiment(R_user_resp[R_user_resp[ent.title() + '_flag'] == True], ent)
    
    for senti in ['pos', 'neg']:
        
        if(senti == 'pos'):
            dataL = l_df_pos
            dataR = r_df_pos

        else:
            dataL = l_df_neg
            dataR = r_df_neg
            
        for moral_cat in range(11):

            # n = min(l_df_neg.shape[0], r_df_neg.shape[0])
            dist1 = dataL[str(moral_cat)].dropna()
            dist2 = dataR[str(moral_cat)].dropna()

    #             n = min(dist1.shape[0], dist2.shape[0])

    #             dist1 = dist1.sample(n)
    #             dist2 = dist2.sample(n)
            # print(dist1.shape, dist2.shape)

            # print(stats.shapiro(dist1), stats.shapiro(dist2))
            # sys.exit()
            if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

                res = get_significance_stats(dist1, dist2, ent, moral_cat, senti)
                # print(res)
                sstat.append(res)

            else:
                print(top, senti, dist1.shape, dist2.shape)
            # sys.exit()
        # print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'Topic', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,0,biden,pos,238107,162911,0.430,0.0,0.408,0.0,0.588,0.294,1,0.000
1,1,biden,pos,238107,162911,0.518,0.0,0.518,0.0,0.000,77.854,1,0.000
2,2,biden,pos,238107,162911,0.143,0.0,0.118,0.0,0.000,444.711,1,0.001
3,3,biden,pos,238107,162911,0.749,0.0,0.741,0.0,0.169,1.888,1,0.000
4,4,biden,pos,238107,162911,0.155,0.0,0.142,0.0,0.000,246.955,1,0.001
5,5,biden,pos,238107,162911,0.405,0.0,0.369,0.0,0.000,141.117,1,0.000
6,6,biden,pos,238107,162911,0.300,0.0,0.285,0.0,0.000,81.837,1,0.000
7,7,biden,pos,238107,162911,0.326,0.0,0.294,0.0,0.000,168.328,1,0.000
8,8,biden,pos,238107,162911,0.068,0.0,0.063,0.0,0.000,189.760,1,0.000
9,9,biden,pos,238107,162911,0.206,0.0,0.193,0.0,0.000,65.304,1,0.000


### Selective exposure vs cross exposure

In [16]:
user_details = pd.read_csv('/home/ahaque2/backup/code/NewSlant/dataset/user_details.csv')

In [17]:
L_sel_exp = user_details[((user_details.L > 0) & (user_details.R == 0) & (user_details.C == 0))] 
R_sel_exp = user_details[((user_details.L == 0) & (user_details.R > 0) & (user_details.C == 0))]

cross_exp = user_details[((user_details.L_f > 0.3) & (user_details.R_f > 0.3) & (user_details.C == 0))]

L_sel_ur = user_resp_df[user_resp_df.author_id_x.isin(L_sel_exp.author_id)]
R_sel_ur = user_resp_df[user_resp_df.author_id_x.isin(R_sel_exp.author_id)]

cross_exp_ur = user_resp_df[user_resp_df.author_id_x.isin(cross_exp.author_id)]
L_sel_ur.shape, R_sel_ur.shape, cross_exp_ur.shape

((714668, 32), (159426, 32), (181721, 32))

In [50]:
L_sel_ur = L_sel_ur.drop(columns=['3'])
R_sel_ur = R_sel_ur.drop(columns=['3'])
cross_exp_ur = cross_exp_ur.drop(columns=['3'])

KeyError: "['3'] not found in axis"

In [56]:
cross_exp_ur['0'].shape

(466953,)

In [20]:
sstat = []
for ms in range(0,11):
    
    if(ms != 3):
        # print(ms)
    
        dist1 = L_sel_ur[str(ms)]
        dist2 = R_sel_ur[str(ms)]
        
        # dist1 = pd.concat((dist1, dist2), axis = 0)
        # dist2 = cross_exp_ur[str(ms)]
        
        # print(dist1.shape)
        # sys.exit()
        if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

            res = get_significance_stats(dist1, dist2, ms, ms, ms)
            # print(res)
            sstat.append(res)

        else:
            print(ms, ms, dist1.shape, dist2.shape)
        # sys.exit()
        # print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'Topic', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,0,0,0,714668,159426,0.181,0.0,0.155,0.0,0.000,815.127,1,0.001
1,1,1,1,714668,159426,0.854,0.0,0.845,0.0,0.000,163.609,1,0.000
2,2,2,2,714668,159426,0.703,0.0,0.687,0.0,0.000,307.332,1,0.000
3,4,4,4,714668,159426,0.887,0.0,0.888,0.0,0.000,608.284,1,0.001
4,5,5,5,714668,159426,0.265,0.0,0.259,0.0,0.000,1289.483,1,0.001
5,6,6,6,714668,159426,0.692,0.0,0.629,0.0,0.000,532.667,1,0.001
6,7,7,7,714668,159426,0.546,0.0,0.534,0.0,0.000,97.476,1,0.000
7,8,8,8,714668,159426,0.318,0.0,0.291,0.0,0.002,9.933,1,0.000
8,9,9,9,714668,159426,0.193,0.0,0.169,0.0,0.000,130.104,1,0.000
9,10,10,10,714668,159426,0.491,0.0,0.485,0.0,0.000,13.886,1,0.000


In [228]:
topics = mft_id_to_label_mapping = pd.read_csv('dataset/MFT_label_to_id_mapping.csv')

mft_id_to_label_mapping = mft_id_to_label_mapping.rename(columns = {'label_id': 'senti'})
mft_id_to_label_mapping

stats_df.merge(mft_id_to_label_mapping, on = 'senti')

# .to_csv('results/statistical_tests/L_vs_R_user_resp_MF.csv')

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES,Unnamed: 0,label
0,0,0,0,714668,181721,0.449,0.0,0.448,0.0,0.352,0.866,1,0.000,2,subversion
1,1,1,1,714668,181721,0.541,0.0,0.553,0.0,0.000,14.729,1,0.000,19,cheating
2,2,2,2,714668,181721,0.150,0.0,0.124,0.0,0.000,231.044,1,0.000,24,care
3,4,4,4,714668,181721,0.161,0.0,0.158,0.0,0.000,67.696,1,0.000,18,fairness
4,5,5,5,714668,181721,0.429,0.0,0.424,0.0,0.008,6.972,1,0.000,7,betrayal
5,6,6,6,714668,181721,0.350,0.0,0.319,0.0,0.000,466.275,1,0.001,4,harm
6,7,7,7,714668,181721,0.328,0.0,0.316,0.0,0.000,49.437,1,0.000,8,authority
7,8,8,8,714668,181721,0.068,0.0,0.057,0.0,0.000,461.271,1,0.001,95,purity
8,9,9,9,714668,181721,0.195,0.0,0.191,0.0,0.176,1.830,1,0.000,3,loyalty
9,10,10,10,714668,181721,0.124,0.0,0.123,0.0,0.000,61.632,1,0.000,1,degradation


### Selective-vs-cross (topic-wise)

In [21]:
sstat = []
topics = list(set(L_sel_ur.theme).intersection(set(R_sel_ur.theme)))
for top in topics:
    
    for ms in range(0,11):

        if(ms != 3):
            # print(ms)

            dist1 = L_sel_ur[L_sel_ur.theme == top][str(ms)]
            dist2 = R_sel_ur[R_sel_ur.theme == top][str(ms)]
            dist1 = pd.concat((dist1, dist2), axis = 0)
            
            dist2 = cross_exp_ur[cross_exp_ur.theme == top][str(ms)]

            # dist2 = L_user_resp[str(ms)].dropna()
            # dist2 = R_user_resp[str(ms)].dropna()
            # dist2 = C_user_resp[str(ms)].dropna()

            # print(dist1.shape)
            # sys.exit()
            if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

                res = get_significance_stats(dist1, dist2, top, ms, ms)
                # print(res)
                sstat.append(res)

            else:
                print(ms, ms, dist1.shape, dist2.shape)
            # sys.exit()
            # print()

stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'Topic', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df    

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,0,georgia,0,7580,2817,0.146,0.0,0.133,0.0,0.146,2.112,1,0.000
1,1,georgia,1,7580,2817,0.871,0.0,0.870,0.0,0.237,1.401,1,0.000
2,2,georgia,2,7580,2817,0.745,0.0,0.753,0.0,0.005,7.782,1,0.001
3,4,georgia,4,7580,2817,0.872,0.0,0.871,0.0,0.085,2.975,1,0.000
4,5,georgia,5,7580,2817,0.298,0.0,0.363,0.0,0.133,2.261,1,0.000
5,6,georgia,6,7580,2817,0.686,0.0,0.590,0.0,0.200,1.645,1,0.000
6,7,georgia,7,7580,2817,0.539,0.0,0.579,0.0,0.635,0.226,1,0.000
7,8,georgia,8,7580,2817,0.327,0.0,0.307,0.0,0.861,0.031,1,0.000
8,9,georgia,9,7580,2817,0.168,0.0,0.160,0.0,0.050,3.848,1,0.000
9,10,georgia,10,7580,2817,0.463,0.0,0.462,0.0,0.228,1.455,1,0.000


In [162]:
stats_df[stats_df.ES > 0.001]

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
3,4,national,4,11328,1633,0.187,0.0,0.135,0.0,0.0,22.978,1,0.002
9,10,national,10,11328,1633,0.124,0.0,0.177,0.0,0.0,40.515,1,0.003
10,0,economy,0,13859,2328,0.430,0.0,0.492,0.0,0.0,34.022,1,0.002
14,5,economy,5,13859,2328,0.447,0.0,0.449,0.0,0.0,27.759,1,0.002
16,7,economy,7,13859,2328,0.305,0.0,0.330,0.0,0.0,26.113,1,0.002
18,9,economy,9,13859,2328,0.206,0.0,0.226,0.0,0.0,34.739,1,0.002
36,7,covid,7,46235,8211,0.274,0.0,0.328,0.0,0.0,136.360,1,0.003
37,8,covid,8,46235,8211,0.070,0.0,0.072,0.0,0.0,85.689,1,0.002
38,9,covid,9,46235,8211,0.176,0.0,0.167,0.0,0.0,131.163,1,0.002
42,2,court,2,12624,1368,0.161,0.0,0.121,0.0,0.0,25.799,1,0.002


In [144]:
topics = set(R_user_resp.theme).intersection(set(L_user_resp.theme))

In [22]:
topic_pairs = []
for top1 in list(topics)[:-1]:
    
    for top2 in list(topics)[0:]:
        
        topic_pairs.append((top1, top2))
        
# topic_pairs  

In [31]:
len(topic_pairs)

156

### Across topic comparison

In [32]:
sstat = []

topics = set(R_user_resp.topics_x).intersection(set(L_user_resp.topics_x))

# for top1 in topics[:-1]:
    
#     for top2 in topics[0:]:

for i, top_p in enumerate(topic_pairs):
    
    top1 = top_p[0]
    top2 = top_p[1]

    for ms in range(0,11):

        if(ms != 3):
            # print(ms)

            # dist1 = L_sel_ur[str(ms)]
            # dist1 = R_sel_ur[str(ms)]
            dist1 = L_user_resp[L_user_resp.theme == top1][str(ms)].dropna()

            # dist2 = L_user_resp[str(ms)].dropna()
            # dist2 = R_user_resp[str(ms)].dropna()
            dist2 = R_user_resp[R_user_resp.theme == top2][str(ms)].dropna()
            
            
#             if(dist2.shape[0] > dist1.shape[0]):
#                 dist1 = dist1.sample(dist1.shape[0])
#                 # dist2 = dist2.sample(10000)
#             else:
                
#                 dist2 = dist2.sample(dist1.shape[0])

            # print(dist1.shape, dist2.shape)
            # sys.exit()
            if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

                res = get_significance_stats(dist1, dist2, top1, top2, ms)
                # print(res)
                sstat.append(res)

            else:
                print(ms, ms, dist1.shape, dist2.shape)
            # sys.exit()
            print(i, end = '\r')
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'Topic', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,georgia,georgia,0,24989,14213,0.143,0.0,0.132,0.0,0.183,1.773,1,0.000
1,georgia,georgia,1,24989,14213,0.863,0.0,0.874,0.0,0.000,42.097,1,0.001
2,georgia,georgia,2,24989,14213,0.747,0.0,0.754,0.0,0.016,5.760,1,0.000
3,georgia,georgia,4,24989,14213,0.871,0.0,0.873,0.0,0.948,0.004,1,0.000
4,georgia,georgia,5,24989,14213,0.283,0.0,0.280,0.0,0.000,12.669,1,0.000
5,georgia,georgia,6,24989,14213,0.633,0.0,0.646,0.0,0.002,9.241,1,0.000
6,georgia,georgia,7,24989,14213,0.528,0.0,0.558,0.0,0.000,38.628,1,0.001
7,georgia,georgia,8,24989,14213,0.319,0.0,0.311,0.0,0.617,0.251,1,0.000
8,georgia,georgia,9,24989,14213,0.170,0.0,0.162,0.0,0.060,3.535,1,0.000
9,georgia,georgia,10,24989,14213,0.449,0.0,0.465,0.0,0.000,20.100,1,0.001


In [153]:
stats_df[stats_df.ES > 0.02]

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
17,economy,national,8,55081,75805,0.069,0.0,0.049,0.0,0.0,3685.903,1,0.028
19,economy,national,10,55081,75805,0.133,0.0,0.105,0.0,0.0,3175.718,1,0.024
137,national,economy,8,75805,55081,0.049,0.0,0.069,0.0,0.0,3685.903,1,0.028
139,national,economy,10,75805,55081,0.105,0.0,0.133,0.0,0.0,3175.718,1,0.024
177,court,economy,8,75805,56500,0.049,0.0,0.079,0.0,0.0,3527.893,1,0.027
205,racism,economy,6,75805,99502,0.257,0.0,0.392,0.0,0.0,5703.507,1,0.033
207,racism,economy,8,75805,99502,0.049,0.0,0.068,0.0,0.0,4433.302,1,0.025
209,racism,economy,10,75805,99502,0.105,0.0,0.144,0.0,0.0,5910.494,1,0.034
256,impeachment,economy,7,75805,52294,0.296,0.0,0.409,0.0,0.0,2674.800,1,0.021
257,impeachment,economy,8,75805,52294,0.049,0.0,0.075,0.0,0.0,2732.808,1,0.021


### Regular-vs-non-regular users [overall]

In [177]:
user_resp_counts_df = user_resp_df.author_id_x.value_counts()

In [239]:
regular_users = user_resp_counts_df[user_resp_counts_df > 50].index
non_regular_users = user_resp_counts_df[user_resp_counts_df == 1].index

In [240]:
reg_df = user_resp_df[user_resp_df.author_id_x.isin(regular_users)]
non_reg_df = user_resp_df[user_resp_df.author_id_x.isin(non_regular_users)]

reg_df.shape, non_reg_df.shape

((1094473, 31), (414390, 31))

In [241]:
sstat = []
for ms in range(0,11):
    
    if(ms != 3):
        # print(ms)
    
        # dist1 = L_sel_ur[str(ms)]
        dist1 = reg_df[str(ms)]
        dist2 = non_reg_df[str(ms)]
        
        # print(dist1.shape)
        # sys.exit()
        if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

            res = get_significance_stats(dist1, dist2, ms, ms, ms)
            # print(res)
            sstat.append(res)

        else:
            print(ms, ms, dist1.shape, dist2.shape)
        # sys.exit()
        # print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'Topic', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df
        

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,0,0,0,1094473,414390,0.468,0.0,0.387,0.0,0.0,16285.356,1,0.011
1,1,1,1,1094473,414390,0.573,0.0,0.469,0.0,0.0,19305.883,1,0.013
2,2,2,2,1094473,414390,0.138,0.0,0.133,0.0,0.0,8138.960,1,0.005
3,4,4,4,1094473,414390,0.166,0.0,0.128,0.0,0.0,18054.881,1,0.012
4,5,5,5,1094473,414390,0.446,0.0,0.369,0.0,0.0,13137.727,1,0.009
5,6,6,6,1094473,414390,0.343,0.0,0.310,0.0,0.0,611.304,1,0.000
6,7,7,7,1094473,414390,0.328,0.0,0.289,0.0,0.0,4687.552,1,0.003
7,8,8,8,1094473,414390,0.060,0.0,0.071,0.0,0.0,1019.966,1,0.001
8,9,9,9,1094473,414390,0.192,0.0,0.177,0.0,0.0,117.276,1,0.000
9,10,10,10,1094473,414390,0.118,0.0,0.130,0.0,0.0,92.607,1,0.000


### Regular-vs-non-regular users [L-vs-R]

In [242]:
L_reg_users = user_details[((user_details.L > 100) & (user_details.R == 0) & (user_details.C == 0))] 
R_reg_users = user_details[((user_details.L == 0) & (user_details.R > 100) & (user_details.C == 0))]

L_non_regular = user_details[((user_details.L <= 100) & (user_details.R == 0) & (user_details.C == 0))] 
R_non_regular = user_details[((user_details.L == 0) & (user_details.R <= 100) & (user_details.C == 0))] 

L_reg_users_df = user_resp_df[user_resp_df.author_id_x.isin(L_reg_users.author_id)]
R_reg_users_df = user_resp_df[user_resp_df.author_id_x.isin(R_reg_users.author_id)]

L_non_reg_users_df = user_resp_df[user_resp_df.author_id_x.isin(L_non_regular.author_id)]
R_non_reg_users_df = user_resp_df[user_resp_df.author_id_x.isin(R_non_regular.author_id)]

L_reg_users_df.shape, R_reg_users_df.shape, L_non_reg_users_df.shape, R_non_reg_users_df.shape

((177284, 31), (17192, 31), (539632, 31), (142234, 31))

In [243]:
sstat = []
for ms in range(0,11):
    
    if(ms != 3):
        # print(ms)
    
        # dist1 = L_sel_ur[str(ms)]
        dist1 = R_non_reg_users_df[str(ms)]
        dist2 = R_reg_users_df[str(ms)]
        
        # print(dist1.shape)
        # sys.exit()
        if(dist1.shape[0] > 10 and dist2.shape[0] > 10):

            res = get_significance_stats(dist1, dist2, ms, ms, ms)
            # print(res)
            sstat.append(res)

        else:
            print(ms, ms, dist1.shape, dist2.shape)
        # sys.exit()
        # print()
    
stats_df = pd.DataFrame(sstat, columns = ['senti', 'Entity', 'Topic', 'Sample size 1', 'sample size 2', 'shapiro_t_dist1', 'shapiro_p_dist1', 'shapiro_t_dist2', 'shapiro_p_dist2', 'KW-p', 'KW-H', 'KW-dof', 'ES'])
stats_df = stats_df.round(3)
stats_df
        

,senti,Entity,Topic,Sample size 1,sample size 2,shapiro_t_dist1,shapiro_p_dist1,shapiro_t_dist2,shapiro_p_dist2,KW-p,KW-H,KW-dof,ES
0,0,0,0,142234,17192,0.429,0.0,0.455,0.0,0.000,48.221,1,0.000
1,1,1,1,142234,17192,0.514,0.0,0.549,0.0,0.000,124.231,1,0.001
2,2,2,2,142234,17192,0.122,0.0,0.125,0.0,0.002,9.283,1,0.000
3,4,4,4,142234,17192,0.147,0.0,0.164,0.0,0.000,121.697,1,0.001
4,5,5,5,142234,17192,0.400,0.0,0.414,0.0,0.000,15.483,1,0.000
5,6,6,6,142234,17192,0.316,0.0,0.326,0.0,0.000,14.566,1,0.000
6,7,7,7,142234,17192,0.310,0.0,0.326,0.0,0.171,1.875,1,0.000
7,8,8,8,142234,17192,0.065,0.0,0.050,0.0,0.000,137.214,1,0.001
8,9,9,9,142234,17192,0.180,0.0,0.186,0.0,0.000,28.395,1,0.000
9,10,10,10,142234,17192,0.142,0.0,0.112,0.0,0.000,103.467,1,0.001
